In [1]:
import pandas as pd
from sqlalchemy import *
import os
import gzip
import json
from ast import literal_eval

from datetime import datetime

In [24]:
for i in os.listdir():
    if 'gz' in i:  
        
        content = list()
        
        with gzip.open(i, 'rb') as zip_file:
            for line in zip_file:
                if line:
                    content.append(json.loads(line))
                    
        with open(i.replace('.gz',''),'w') as json_file:
            json.dump(content, json_file)

In [2]:
receipts = pd.read_json('receipts.json')
brands = pd.read_json('brands.json')
users = pd.read_json('users.json')

### Receipt Table

In [4]:
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [25]:
receipts.isnull().any()

_id                        False
bonusPointsEarned           True
bonusPointsEarnedReason     True
createDate                 False
dateScanned                False
finishedDate                True
modifyDate                 False
pointsAwardedDate           True
pointsEarned                True
purchaseDate                True
purchasedItemCount          True
rewardsReceiptItemList      True
rewardsReceiptStatus       False
totalSpent                  True
userId                     False
dtype: bool

After browsing the receipt data, the id, date and rewardsReceiptItemList columns would need some transformation. These columns have dictionary in the cells and the next step will check how many keys in each column. This will determin how many columns should be generated after the transformation. 

In [28]:
pd.DataFrame([x.keys() for x in receipts['_id']])[0].unique()

array(['$oid'], dtype=object)

In [29]:
pd.DataFrame([x.keys() for x in receipts['createDate']])[0].unique()

array(['$date'], dtype=object)

In [30]:
pd.DataFrame([x.keys() for x in receipts['dateScanned']])[0].unique()

array(['$date'], dtype=object)

In [31]:
pd.DataFrame([x.keys() for x in receipts.dropna()['finishedDate']])[0].unique()

array(['$date'], dtype=object)

In [32]:
pd.DataFrame([x.keys() for x in receipts['modifyDate']])[0].unique()

array(['$date'], dtype=object)

In [33]:
pd.DataFrame([x.keys() for x in receipts.dropna()['pointsAwardedDate']])[0].unique()

array(['$date'], dtype=object)

In [34]:
pd.DataFrame([x.keys() for x in receipts.dropna()['purchaseDate']])[0].unique()

array(['$date'], dtype=object)

In [35]:
receipts.dropna()['rewardsReceiptItemList']
## this column has more than one key in the dictionary format. Will transform this later.

0       [{'barcode': '4011', 'description': 'ITEM NOT ...
1       [{'barcode': '4011', 'description': 'ITEM NOT ...
3       [{'barcode': '4011', 'description': 'ITEM NOT ...
4       [{'barcode': '4011', 'description': 'ITEM NOT ...
5       [{'barcode': '4011', 'description': 'ITEM NOT ...
                              ...                        
971     [{'barcode': '4011', 'description': 'ITEM NOT ...
984     [{'barcode': '079400066619', 'competitiveProdu...
985     [{'barcode': '4011', 'description': 'ITEM NOT ...
990     [{'barcode': '4011', 'description': 'ITEM NOT ...
1021    [{'barcode': '665290001184', 'description': 'B...
Name: rewardsReceiptItemList, Length: 454, dtype: object

#### Transform ID and date related columns

In [36]:
receipts['createDate'].apply(lambda x: x.get(list(x.keys())[0]))

0       1609687531000
1       1609687483000
2       1609687537000
3       1609687534000
4       1609687506000
            ...      
1114    1614594147000
1115    1614613361873
1116    1614607657664
1117    1614604048000
1118    1614565354962
Name: createDate, Length: 1119, dtype: int64

It looks like time are stored in second so would need to transfer into UTC format
    

In [3]:
def date_convert(date):
    
    try:
        return(datetime.utcfromtimestamp(int(date['$date'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
    except:
        return(None)

In [4]:
receipts['_id'] = receipts['_id'].apply(lambda x: x['$oid'])
receipts['createDate'] = receipts['createDate'].apply(lambda x: date_convert(x))
receipts['dateScanned'] = receipts['dateScanned'].apply(lambda x: date_convert(x))
receipts['finishedDate'] = receipts['finishedDate'].apply(lambda x: date_convert(x))
receipts['modifyDate'] = receipts['modifyDate'].apply(lambda x: date_convert(x))
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].apply(lambda x: date_convert(x))
receipts['purchaseDate'] = receipts['purchaseDate'].apply(lambda x: date_convert(x))

In [5]:
receipts = receipts.explode('rewardsReceiptItemList')
receipts.reset_index(inplace=True)

receipts

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"{'barcode': '028400642255', 'description': 'DO...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
3,2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,None,2021-01-03 15:25:42,None,5.0,2021-01-03 00:00:00,1.0,"{'needsFetchReview': False, 'partnerItemId': '...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
4,3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41,2021-03-01 15:42:41,None,2021-03-01 15:42:41,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
7377,1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37,2021-03-01 14:07:37,None,2021-03-01 14:07:37,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
7378,1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,None,2021-03-01 13:07:29,None,25.0,2020-08-17 00:00:00,2.0,"{'barcode': 'B076FJ92M4', 'description': 'muel...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
7379,1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,None,2021-03-01 13:07:29,None,25.0,2020-08-17 00:00:00,2.0,"{'barcode': 'B07BRRLSVC', 'description': 'thin...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [6]:
receipts = receipts.fillna({'rewardsReceiptItemList':'{}'}) ## fill NA cells resulted from exploding with empty dictionary


In [84]:
 ## convert each cell to string because the next step is to use literal_eval function
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(lambda x:str(x))


receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(literal_eval)




In [8]:
receipts_norm = pd.json_normalize(receipts['rewardsReceiptItemList'],errors='ignore',record_prefix='rewardsReceiptItemList').add_prefix('rewardsReceiptItemList.')

receipts_norm

,rewardsReceiptItemList.barcode,rewardsReceiptItemList.description,rewardsReceiptItemList.finalPrice,rewardsReceiptItemList.itemPrice,rewardsReceiptItemList.needsFetchReview,rewardsReceiptItemList.partnerItemId,rewardsReceiptItemList.preventTargetGapPoints,rewardsReceiptItemList.quantityPurchased,rewardsReceiptItemList.userFlaggedBarcode,rewardsReceiptItemList.userFlaggedNewItem,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
7379,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


In [69]:
receipts_final = pd.merge(receipts, receipts_norm, left_index=True, right_index=True, how='inner')

receipts_final

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,None,2021-01-03 15:25:42,None,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41,2021-03-01 15:42:41,None,2021-03-01 15:42:41,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37,2021-03-01 14:07:37,None,2021-03-01 14:07:37,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,None,2021-03-01 13:07:29,None,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
7379,1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,None,2021-03-01 13:07:29,None,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


### Brands Table

In [12]:
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [13]:
brands.isnull().any()

_id             False
barcode         False
category         True
categoryCode     True
cpg             False
name            False
topBrand         True
brandCode        True
dtype: bool

Perform the same data manuplation on id and cpg column.

In [5]:
pd.DataFrame([x.keys() for x in brands['_id']])[0].unique()

array(['$oid'], dtype=object)

In [6]:
pd.DataFrame([x.keys() for x in brands['cpg']])[0].unique()

array(['$id', '$ref'], dtype=object)

There are 2 different keys in cpg column. Will need to explode on this.

In [17]:
brands_norm = pd.json_normalize(brands['cpg']).add_prefix('cpg.')

brands_norm

,cpg.$ref,cpg.$id.$oid
0,Cogs,601ac114be37ce2ead437550
1,Cogs,5332f5fbe4b03c9a25efd0ba
2,Cogs,601ac142be37ce2ead437559
3,Cogs,601ac142be37ce2ead437559
4,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...
1162,Cogs,5f77274dbe37ce6b592e90bf
1163,Cogs,53e10d6368abd3c7065097cc
1164,Cogs,5332fa12e4b03c9a25efd1e7
1165,Cogs,5332f5f6e4b03c9a25efd0b4


In [18]:
brands_final = pd.merge(brands, brands_norm, left_index=True, right_index=True, how='outer')

brands_final

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,cpg.$ref,cpg.$id.$oid
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...,...,...,...,...,...,...,...,...
1162,{'$oid': '5f77274dbe37ce6b592e90c0'},511111116752,Baking,BAKING,"{'$ref': 'Cogs', '$id': {'$oid': '5f77274dbe37...",test brand @1601644365844,NaN,NaN,Cogs,5f77274dbe37ce6b592e90bf
1163,{'$oid': '5dc1fca91dda2c0ad7da64ae'},511111706328,Breakfast & Cereal,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '53e10d6368ab...",Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,Cogs,53e10d6368abd3c7065097cc
1164,{'$oid': '5f494c6e04db711dd8fe87e7'},511111416173,Candy & Sweets,CANDY_AND_SWEETS,"{'$ref': 'Cogs', '$id': {'$oid': '5332fa12e4b0...",test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,Cogs,5332fa12e4b03c9a25efd1e7
1165,{'$oid': '5a021611e4b00efe02b02a57'},511111400608,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5f6e4b0...",LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,Cogs,5332f5f6e4b03c9a25efd0b4


In [19]:
brands_final['_id'] = brands_final['_id'].apply(lambda x: x['$oid'])

In [20]:
brands_final.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,cpg.$ref,cpg.$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7


### User Table

In [22]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [23]:
users.isnull().any()

_id             False
active          False
createdDate     False
lastLogin        True
role            False
signUpSource     True
state            True
dtype: bool

In [21]:
pd.DataFrame([x.keys() for x in users['_id']])[0].unique()

array(['$oid'], dtype=object)

In [61]:
pd.DataFrame([x.keys() for x in users['createdDate']])[0].unique()

array(['$date'], dtype=object)

In [66]:
pd.DataFrame([x.keys() for x in users.dropna()['lastLogin']])[0].unique()

array(['$date'], dtype=object)

In [24]:
users['createdDate'] = users['createdDate'].apply(lambda x: date_convert(x))
users['lastLogin'] = users['lastLogin'].apply(lambda x: date_convert(x))

In [25]:
users['_id'] = users['_id'].apply(lambda x: x['$oid'])

In [26]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


In [37]:
users['_id'].dropna().apply(len).describe()

count    495.0
mean      24.0
std        0.0
min       24.0
25%       24.0
50%       24.0
75%       24.0
max       24.0
Name: _id, dtype: float64

In [43]:
receipts_final['dateScanned'].max()

'2021-03-01 23:17:34'

### Write 3 tables to csv files

In [71]:
receipts_final.to_csv('receipts.csv')
brands_final.to_csv('brands.csv')
users.to_csv('users.csv')

In [20]:
receipts_final.to_excel('receipts.xlsx')
brands_final.to_excel('brands.xlsx')
users.to_excel('users.xlsx')

#### Q1: What are the top 5 brands by receipts scanned for most recent month?

In [ ]:

WITH D AS (
SELECT MAX(dateScanned) AS LAST_D
FROM RECEIPTS
),
SCAN AS (
SELECT B.NAME,
       COUNT(*) NUMBER_OF_SCAN
FROM RECEIPTS R
INNER JOIN BRANDS B
    ON R.BRANDCODE = B.BRANDCODE
    AND R.BARCODE = B.BARCODE
INNER JOIN D
ON ADD_MONTHS(LAST_D, -1) AND LAST_D
GROUP BY B.NAME
ORDER BY COUNT(*) DESC
)
SELECT *
FROM SCAN
FETCH NEXT 5 ROWS ONLY


#### Q2: How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [ ]:

WITH D AS (
SELECT MAX(dateScanned) AS LAST_D
FROM RECEIPTS
),
SCAN_1 AS (
SELECT B.NAME,
       1 AS FLAG
       COUNT(*) NUMBER_OF_SCAN
FROM RECEIPTS R
INNER JOIN BRANDS B
    ON R.BRANDCODE = B.BRANDCODE
    AND R.BARCODE = B.BARCODE
INNER JOIN D
ON ADD_MONTHS(LAST_D, -1) AND LAST_D
GROUP BY B.NAME, FLAG
ORDER BY COUNT(*) DESC
FETCH NEXT 5 ROWS ONLY
),
SCAN_2 AS(
SELECT B.NAME,
       2 AS FLAG,
       COUNT(*) NUMBER_OF_SCAN
FROM RECEIPTS R
INNER JOIN BRANDS B
    ON R.BRANDCODE = B.BRANDCODE
    AND R.BARCODE = B.BARCODE
INNER JOIN D
ON ADD_MONTHS(LAST_D, -2)-1 AND ADD_MONTHS(LAST_D, -1) -1
GROUP BY B.NAME, FLAG
ORDER BY COUNT(*) DESC
FETCH NEXT 5 ROWS ONLY
)
SELECT *
FROM SCAN_1
UNION
SELECT *
FROM SCAN_2



#### Q3: When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

Without knowing how the receipt validation process, this question can't be answered properly. If I have to guess, I would think the 'Accepted' is better. At least the accepted receipts mean the spending is valid. 

In [12]:
receipts_final.groupby('rewardsReceiptStatus')['totalSpent'].mean()

rewardsReceiptStatus
FINISHED     1244.372934
FLAGGED      2635.570247
PENDING        28.032449
REJECTED       19.544970
SUBMITTED            NaN
Name: totalSpent, dtype: float64

#### Q4: When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

If accepted rewards status means the transaction is valid, it would be better to be considered for total number of items.

#### Q5: Which brand has the most spend among users who were created within the past 6 months?

In [ ]:
WITH USER6M AS (
SELECT *
FROM USERS 
WHERE CREATEDDATE BETWEEN ADD_MONTHS(SYSDATE, -6) AND SYSDATE
)
SELECT B.NAME,
       SUM(TOT)
FROM RECEIPTS R
INNER JOIN USER6M U
    ON R.USERID = U.USERID
INNER JOIN BRANDS B
    ON R.BRANDCODE = B.BRANDCODE
    AND R.BARCODE = B.BARCODE
GROUP BY B.NAME
ORDER BY 2 DESC
FETCH NEXT 1 ROWS ONLY;

#### Q6: Which brand has the most transactions among users who were created within the past 6 months?

In [ ]:
WITH USER6M AS (
SELECT *
FROM USERS 
WHERE CREATEDDATE BETWEEN ADD_MONTHS(SYSDATE, -6) AND SYSDATE
)
SELECT B.NAME,
       COUNT(DISTINCT R.ID)
FROM RECEIPTS R
INNER JOIN USER6M U
    ON R.USERID = U.USERID
INNER JOIN BRANDS B
    ON R.BRANDCODE = B.BRANDCODE
    AND R.BARCODE = B.BARCODE
GROUP BY B.NAME
ORDER BY 2 DESC
FETCH NEXT 1 ROWS ONLY;

### Third: Evaluate Data Quality Issues in the Data Provided

In [15]:
receipts_final['rewardsReceiptItemList.brandCode'].nunique()

227

In [21]:
brands_final['brandCode'].nunique()

897

In [44]:
brand_cross = pd.merge(receipts_final, brands_final, how='left', left_on='rewardsReceiptItemList.brandCode', right_on='brandCode' )[['rewardsReceiptItemList.brandCode', 'brandCode']].drop_duplicates()

In [55]:
brand_cross[(brand_cross['rewardsReceiptItemList.brandCode'].notna() )& (brand_cross['brandCode'].isnull())].sort_values(by =['rewardsReceiptItemList.brandCode'])

,rewardsReceiptItemList.brandCode,brandCode
720986,7UP,NaN
167949,ADVIL,NaN
797916,AMERICAN BEAUTY,NaN
727332,ARROWHEAD,NaN
301504,AZTECA,NaN
...,...,...
530641,WELCH'S,NaN
76758,WINGSTOP,NaN
529230,WISHBONE,NaN
168659,WONDERFUL,NaN


Above table includes brands code on the receipts that are not in the brands table.

In [ ]:
receipt_n = receipts_final.dropna()

brands_n = brands_final.dropna()

In [27]:
users.describe()

,_id,active,createdDate,lastLogin,role,signUpSource,state
count,495,495,495,433,495,447,439
unique,212,2,212,171,2,2,8
top,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,consumer,Email,WI
freq,20,494,20,20,413,443,396


In [28]:
users['_id'].nunique()

212

In [40]:
sum(users.duplicated())

283

There are duplicated records in the user table which requires some cleanup.

In [64]:
receipts_final[receipts_final['rewardsReceiptItemList.description'] == 'ITEM NOT FOUND']['_id'].nunique()

165

In [76]:
receipts_final[[
               'rewardsReceiptItemList.pointsPayerId',
               'rewardsReceiptItemList.rewardsProductPartnerId',
                'rewardsReceiptItemList.rewardsGroup',
               'rewardsReceiptItemList.metabriteCampaignId']].dropna()

,rewardsReceiptItemList.pointsPayerId,rewardsReceiptItemList.rewardsProductPartnerId,rewardsReceiptItemList.rewardsGroup,rewardsReceiptItemList.metabriteCampaignId
308,5332f5fbe4b03c9a25efd0ba,5332f5fbe4b03c9a25efd0ba,PEPSI 12 OZ 12 PACK,PEPSI 12 OZ 12 PACK
310,5332f5fbe4b03c9a25efd0ba,5332f5fbe4b03c9a25efd0ba,DORITOS NACHO CHEESE MULTI SERVE,DORITOS NACHO CHEESE MULTI SERVE
311,550b2565e4b001d5e9e4146f,550b2565e4b001d5e9e4146f,KLEENEX TRUSTED CARE FACIAL TISSUES 120 - 179 ...,KLEENEX TRUSTED CARE FACIAL TISSUES 120 - 179 ...
637,5332f5f6e4b03c9a25efd0b4,5332f5f6e4b03c9a25efd0b4,BEN AND JERRYS ICE CREAM,BEN AND JERRYS ICE CREAM
639,550b2565e4b001d5e9e4146f,550b2565e4b001d5e9e4146f,KLEENEX TRUSTED CARE FACIAL TISSUES 120 - 179 ...,KLEENEX TRUSTED CARE FACIAL TISSUES 120 - 179 ...
...,...,...,...,...
6695,550b2565e4b001d5e9e4146f,550b2565e4b001d5e9e4146f,HUGGIES NATURAL CARE SENSITIVE SKIN BABY WIPES...,HUGGIES NATURAL CARE SENSITIVE SKIN BABY WIPES...
6696,550b2565e4b001d5e9e4146f,550b2565e4b001d5e9e4146f,HUGGIES NATURAL CARE SENSITIVE SKIN BABY WIPES...,HUGGIES NATURAL CARE SENSITIVE SKIN BABY WIPES...
6697,550b2565e4b001d5e9e4146f,550b2565e4b001d5e9e4146f,HUGGIES NATURAL CARE SENSITIVE SKIN BABY WIPES...,HUGGIES NATURAL CARE SENSITIVE SKIN BABY WIPES...
6698,550b2565e4b001d5e9e4146f,550b2565e4b001d5e9e4146f,HUGGIES NATURAL CARE SENSITIVE SKIN BABY WIPES...,HUGGIES NATURAL CARE SENSITIVE SKIN BABY WIPES...


In [80]:
users['signUpSource'].unique()

array(['Email', 'Google', nan], dtype=object)

In [83]:
users['state'].unique()

array(['WI', 'KY', 'AL', 'CO', 'IL', nan, 'OH', 'SC', 'NH'], dtype=object)